# MNIST Auto-Encoder
TODO: Test mnist_autoencoder on PyCharm

## Pre-requisites

Install the following Python packages

1. cloudmesh-installer
2. cloudmesh-common

In [1]:
try:
    from cloudmesh.common.StopWatch import StopWatch
except:  # noqa: E722
    ! pip install cloudmesh-common
    from cloudmesh.common.StopWatch import StopWatch

## Exporting Output Graphs

In [ ]:
def save(graph, filename):
    if os.path.exists("images"):
        pass
    else:
        Shell.mkdir("images")
    plot_model(graph, to_file=f'images/{filename}.png', show_shapes=True)
    plot_model(graph, to_file=f'images/{filename}.pdf', show_shapes=True)

## Import Libraries

In [ ]:
StopWatch.start("total")
StopWatch.start("import")
StopWatch.progress(0)

import os    # noqa: E402
import cpuinfo    # noqa: E402
import numpy as np    # noqa: E402
import matplotlib.pyplot as plt    # noqa: E402
from keras.layers import Dense, Input    # noqa: E402
from keras.layers import Conv2D, Flatten    # noqa: E402
from keras.layers import Reshape, Conv2DTranspose    # noqa: E402
from keras.models import Model    # noqa: E402
from keras.datasets import mnist    # noqa: E402
from keras.utils import plot_model    # noqa: E402
from keras import backend as K    # noqa: E402
from cloudmesh.common.systeminfo import os_is_windows    # noqa: E402
from cloudmesh.common.Shell import Shell    # noqa: E402

StopWatch.stop("import")
StopWatch.progress(1)

## Data Load

In [ ]:
StopWatch.start("data-load")

(x_train, y_train), (x_test, y_test) = mnist.load_data()

StopWatch.stop("data-load")
StopWatch.progress(20)

## Data Pre-Process

In [4]:
StopWatch.start("data-pre-process")

image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

input_shape = (image_size, image_size, 1)

StopWatch.stop("data-pre-process")
StopWatch.progress(30)

# cloudmesh status=running progress=20 pid=8968


## Define Model

In [5]:
StopWatch.start("compile")

batch_size = 32
kernel_size = 3
latent_dim = 16
hidden_units = [32, 64]

inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
x = Dense(hidden_units[0], activation='relu')(x)
x = Dense(hidden_units[1], activation='relu')(x)

shape = K.int_shape(x)

# generate latent vector
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

# instantiate encoder model
encoder = Model(inputs,
                latent,
                name='encoder')
encoder.summary()
save(encoder, 'encoder')

latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)
x = Dense(hidden_units[0], activation='relu')(x)
x = Dense(hidden_units[1], activation='relu')(x)

outputs = Dense(1, activation='relu')(x)

decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
save(decoder, 'decoder')

autoencoder = Model(inputs,
                    decoder(encoder(inputs)),
                    name='autoencoder')
autoencoder.summary()
save(autoencoder, 'autoencoder')

autoencoder.compile(loss='mse', optimizer='adam')

StopWatch.stop('compile')
StopWatch.progress(40)

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 dense (Dense)               (None, 28, 28, 32)        64        
                                                                 
 dense_1 (Dense)             (None, 28, 28, 64)        2112      
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 latent_vector (Dense)       (None, 16)                802832    
                                                                 
Total params: 805,008
Trainable params: 805,008
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) and inst

'# cloudmesh status=running progress=40 pid=8968'

## Train

In [6]:
StopWatch.start("train")

autoencoder.fit(x_train,
                x_train,
                validation_data=(x_test, x_test),
                epochs=1,
                batch_size=batch_size)

StopWatch.stop("train")
StopWatch.progress(60)

 404/1875 [=====>........................] - ETA: 38s - loss: 0.0284


KeyboardInterrupt



## Test

In [ ]:
StopWatch.start("test")

x_decoded = autoencoder.predict(x_test)

StopWatch.stop("test")
StopWatch.progress(80)

## Visualize

In [ ]:
StopWatch.start("visualize")

imgs = np.concatenate([x_test[:8], x_decoded[:8]])
imgs = imgs.reshape((4, 4, image_size, image_size))
imgs = np.vstack([np.hstack(i) for i in imgs])
plt.figure()
plt.axis('off')
plt.title('Input: 1st 2 rows, Decoded: last 2 rows')
plt.imshow(imgs, interpolation='none', cmap='gray')
plt.savefig(f'images/input_and_decoded.png',dpi=300)
plt.savefig(f'images/input_and_decoded.pdf')
plt.savefig(f'images/input_and_decoded.svg')

StopWatch.stop("visualize")
StopWatch.stop("total")
StopWatch.progress(100)

if os_is_windows():
    user = os.environ["USERNAME"]
else:
    try:
        user = os.environ['USER']
    except:
        user = os.system('basename $HOME')

try:
    gpuname = ''
    for line in open('mnist_autoencoder.log', 'r'):
        if 'GPU' in line and line[-2] == ')':
            gpuname = gpuname + line[:line.find('(')] + '\n'
except:
    gpuname = cpuinfo.get_cpu_info()['brand_raw']

tag = 'mnist_autoencoder'

StopWatch.benchmark(tag=tag, node=gpuname, user=user)